In [1]:
from data import uci_har, uci_preprocessing
from simulation.make_data import get_dataloader
from pathlib import Path
from models import HARClassifier, VAE_1D, resnet18_1d

from training import har_train, OnManifoldPerturbation_v2, structured_har, structured_resnet

from training.trades import trades_loss

import torch
from torch import nn, optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

from utils import AverageMeter, concatenate, save_data, load_data

from tqdm.auto import tqdm
from torch.optim.lr_scheduler import MultiStepLR
import os

from attacker import LinfPGDAttack

import copy

In [2]:
class LatentClf(nn.Module):
    def __init__(self):
        super(LatentClf, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, 5, padding=2, stride=2)  # in_channels=3
        self.bn1 = nn.BatchNorm1d(64, momentum=0.9)
        self.maxpool = nn.MaxPool1d(2, 2)
        self.conv2 = nn.Conv1d(64, 128, 5, padding=2, stride=2)
        self.bn2 = nn.BatchNorm1d(128, momentum=0.9)
        self.fc1 = nn.Linear(128 * 4, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.LeakyReLU()
        self.bn3 = nn.BatchNorm1d(256, momentum=0.9)

    def forward(self, x):
        out = self.maxpool(self.relu(self.bn1(self.conv1(x))))
        out = self.relu(self.bn2(self.conv2(out)))
#         print(out.shape)
        out = out.view(x.size(0),-1)
        out = self.relu(self.bn3(self.fc1(out)))
        out = self.relu(self.fc2(out))
        return out

In [3]:
def generate_adv(net, test_loader, device, epsilon=1.0, alpha=0.01, k=7):
    net.eval()
    adv = LinfPGDAttack(net, epsilon=epsilon, alpha=alpha, k=k)
    
    advs = None
    correct_ys = None
    total = None
    total_ys = None
    for X, y in test_loader:
        adv_x = adv.perturb(X.to(device), y.long().to(device))
        out = net(adv_x)
        out = F.softmax(out, dim=1)
        _, predicted = out.max(1)
        idx = ~predicted.eq(y.to(device))
        advs = concatenate(advs, adv_x[idx].detach().cpu().numpy())
        correct_ys = concatenate(correct_ys, y[idx].detach().cpu().numpy())
        total = concatenate(total, adv_x.detach().cpu().numpy())
        total_ys = concatenate(total_ys, y.detach().cpu().numpy())
        
    adv_dataloader = get_dataloader(advs, correct_ys, drop_last=False, shuffle=False)
    total_adv_dataloader = get_dataloader(total, total_ys, drop_last=False, shuffle=False)
    
    return adv_dataloader, advs, correct_ys, total_adv_dataloader

def accuracy2(net, test_loader, device):
    net.eval()
    benign_correct = 0
    total_set = 0

    for X, y in test_loader:
        X = X.to(device)
        out = net(X)
        out = F.softmax(out, dim=1)
        _, predicted = out.max(1)
        idx = predicted.eq(y.to(device))
        benign_correct += predicted.eq(y.to(device)).sum().item()

        total_set += X.size(0)

    benign_acc = benign_correct / total_set
    print('accuracy: {0}'.format(benign_acc))
    return benign_acc, benign_correct, total_set


def accuracy(net, test_loader, device, epsilon=1.0, alpha=0.01, k=7):
    net.eval()
    benign_correct = 0
    adv_correct = 0
    total_set = 0
    correct_list = None
    correct_label = None
    adv_list = None
    benign_softmax_list = None
    correct_softmax_list = None
    adversarial_softmax_list = None
    correct_adversarial_softmax_list = None

    adv = LinfPGDAttack(net, epsilon=epsilon, alpha=alpha, k=k)

    for X, y in test_loader:
        X = X.to(device)
        out = net(X)
        out = F.softmax(out, dim=1)
        benign_softmax_list = concatenate(benign_softmax_list, out.detach().cpu().numpy())
        _, predicted = out.max(1)
        idx = predicted.eq(y.to(device))
        benign_correct += predicted.eq(y.to(device)).sum().item()

        correct_softmax_list = concatenate(correct_softmax_list, out[idx].detach().cpu().numpy())
        correct_list = concatenate(correct_list, X[idx].detach().cpu().numpy())
        correct_label = concatenate(correct_label, y[idx].detach().cpu().numpy())

        adv_x = adv.perturb(X.to(device), y.long().to(device))

        perturbation = adv_x - X
        out = net(adv_x)
        out = F.softmax(out, dim=1)
        adversarial_softmax_list = concatenate(adversarial_softmax_list, out.detach().cpu().numpy())
        _, predicted = out.max(1)
        idx = predicted.eq(y.to(device))
        correct_adversarial_softmax_list = concatenate(correct_adversarial_softmax_list, out[idx].detach().cpu().numpy())
        adv_correct += predicted.eq(y.to(device)).sum().item()

        adv_list = concatenate(adv_list, perturbation.detach().cpu().numpy())

        total_set += X.size(0)

    benign_acc = benign_correct / total_set
    adv_acc = adv_correct / total_set
    print('benign accuracy: {0}\tadversarial accuracy: {1}'.format(benign_acc, adv_acc))
    return benign_acc, adv_acc, correct_list, correct_label, adv_list


def on_manifold_generation(classifier, vae, l_clf, loader, device, epsilon=0.3, k=7, alpha=0.0784, filter_=False, test_=False):
    on_adv = OnManifoldPerturbation_v2(classifier, vae, device, eta=epsilon, k=k, alpha=alpha)
    
    on_adv_Xs = None
    on_ys = None
    on_zs = None
    on_original_zs = None
    attack_succ_idxs = None
    orig_X = None
    orig_y = None
    
    print('creating perturbed data..')

    for X, y in loader:
    #     X = X.reshape((X.shape[0], -1))
        z, z_pert, adv_x = on_adv.perturb(X.to(device), y.long().to(device))
        on_adv_Xs = concatenate(on_adv_Xs, adv_x.detach().cpu().numpy())
        on_zs = concatenate(on_zs, z_pert.detach().cpu().numpy())
        on_ys = concatenate(on_ys, y.detach().cpu().numpy())
        on_original_zs  = concatenate(on_original_zs, z.detach().cpu().numpy())
        orig_X = concatenate(orig_X, X.detach().numpy())
        orig_y = concatenate(orig_y, y.detach().numpy())

        f_x = classifier(adv_x)
        f_x = F.softmax(f_x, dim=1)
        _, predicted = f_x.max(1)

        idxs = ~predicted.eq(y.to(device))
        attack_succ_idxs = concatenate(attack_succ_idxs, idxs.detach().cpu().numpy())

    # attack_succ_rate = attack_succ_perturbed.shape[0]/10000
    # print(attack_succ_rate)

    adv_loader = get_dataloader(on_adv_Xs, on_ys)
    z_loader = get_dataloader(on_zs, on_ys)


    on_xs = None
    on_ys = None
    on_preds = None
    
    print('filtering perturbed data..')

    for x_tilde, y in adv_loader:
        out, _, _ = vae.encode(x_tilde.to(device))
        out = l_clf(out.unsqueeze(1))
        out = F.softmax(out, dim=1)
        _, pred = out.max(1)

        idx =  pred.eq(y.to(device))

        on_adv_filtered = x_tilde[idx].detach().cpu().numpy()
        on_adv_filtered_label = y[idx].detach().cpu().numpy()

        on_xs = concatenate(on_xs, on_adv_filtered)
        on_ys = concatenate(on_ys, on_adv_filtered_label)
        on_preds = concatenate(on_preds, pred.detach().cpu().numpy())

    final_loader = get_dataloader(on_xs, on_ys)
#     final_loader = adv_loader
    
    if filter_:

        filtered_onxs = None
        filtered_onys = None

        print('organizing training data..')

        for x_tilde, y in final_loader:
            out = classifier(x_tilde.to(device))
            f_x = F.softmax(out, dim=1)
            _, predicted = f_x.max(1)
            idxs = ~predicted.eq(y.to(device))

            filtered_onxs = concatenate(filtered_onxs, x_tilde[idxs].detach().numpy())
            filtered_onys = concatenate(filtered_onys, y[idxs].detach().numpy())

        if test_:
            final_loader = get_dataloader(filtered_onxs, filtered_onys)
        else:
            total_X = concatenate(orig_X, filtered_onxs)
            total_y = concatenate(orig_y, filtered_onys)

            final_loader = get_dataloader(total_X, total_y)
    
    return final_loader

In [19]:
cuda_num = 4
device = 'cuda:%d'%cuda_num

model_dir='./simulation/HAR_UCI/'
vae = VAE_1D(9, 1536)
state_dict = torch.load(os.path.join(model_dir, 'vae-epoch{}.pt'.format(95)))
vae.load_state_dict(state_dict)
vae.to(device)

model = HARClassifier()
model.to(device)
model.load_state_dict(torch.load(os.path.join(model_dir, 'comp0_0-model-epoch{}.pt'.format(99))))

l_clf = LatentClf()
l_clf.to(device)
l_clf.load_state_dict(torch.load(os.path.join(model_dir, 'l_clf.pt')))

<All keys matched successfully>

### Generate on_adv_perturbation

In [5]:
train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
# onadv_loader = on_manifold_generation(model, vae, l_clf, train_loader, device, epsilon=0.07, k=7, alpha=0.01, filter_=True)

In [148]:
for X, y in onadv_test_loader:
    print(X.shape)

torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([128, 9, 128])
torch.Size([19, 9, 128])


In [6]:
# model_dir='./simulation/HAR_UCI/'
# model = HARClassifier()
# model.to(device)
# model.load_state_dict(torch.load(os.path.join(model_dir, 'comp0_0-model-epoch{}.pt'.format(99))))

# comp_ratio = 0.9

model = resnet18_1d(output_len=6, input_channel=9)
model.to(device)

my_file = Path('./simulation/HAR_UCI/model-epoch49.pt')
model.load_state_dict(torch.load(my_file))
model.eval()
# on_adv_acc = accuracy2(model, onadv_test_loader, device)

ResNet(
  (conv1): Conv1d(9, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 

In [20]:
off_test_loader, _, _, _ = generate_adv(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)

In [21]:
accuracy2(model, off_test_loader, device)

accuracy: 0.0


(0.0, 0, 1912)

In [22]:
epsilon = 0.003
k = 7
alpha = 0.0003

onadv_loader = on_manifold_generation(model, vae, l_clf, train_loader, device, epsilon=epsilon, k=k, alpha=alpha, filter_=True)
onadv_test_loader = on_manifold_generation(model, vae, l_clf, test_loader, device, epsilon=epsilon, k=k, alpha=alpha, filter_=True, test_=True)

creating perturbed data..
filtering perturbed data..
organizing training data..
creating perturbed data..
filtering perturbed data..
organizing training data..


In [209]:
normal_acc, off_acc, on_acc = offadv_comp(0.9, retrain=True, EPOCHs=20)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 5	Train loss: 0.4404	Validation accuracy: 0.8410
Epoch 10	Train loss: 0.3228	Validation accuracy: 0.8707
Epoch 15	Train loss: 0.2695	Validation accuracy: 0.8937
Epoch 20	Train loss: 0.2430	Validation accuracy: 0.9020
benign accuracy: 0.9002375296912114	adversarial accuracy: 0.7974211062097047
accuracy: 0.11278195488721804


In [207]:
normal_acc, off_acc, on_acc = onadv_comp(0.9, onadv_loader, retrain=True, EPOCHs=20)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 5	Train loss: 0.2789	Validation accuracy: 0.8937
Epoch 10	Train loss: 0.2710	Validation accuracy: 0.8887
Epoch 15	Train loss: 0.2596	Validation accuracy: 0.9127
Epoch 20	Train loss: 0.2552	Validation accuracy: 0.9180
benign accuracy: 0.9165252799457075	adversarial accuracy: 0.25517475398710554
accuracy: 0.41604010025062654


In [160]:
epsilon = 0.0005
k = 7
alpha = 0.00005



normal_acc, off_acc, on_acc = onadv_comp(comp_ratio, onadv_loader, retrain=True)

creating perturbed data..
filtering perturbed data..
organizing training data..


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 5	Train loss: 0.6070	Validation accuracy: 0.8192
Epoch 10	Train loss: 0.3977	Validation accuracy: 0.8913
Epoch 15	Train loss: 0.3085	Validation accuracy: 0.8937
benign accuracy: 0.8917543264336614	adversarial accuracy: 0.2480488632507635
accuracy: 0.13202247191011235


In [187]:
comp_ratio = 0.9

model = resnet18_1d(output_len=6, input_channel=9)
model.to(device)

my_file = Path('./simulation/HAR_UCI/model-epoch49.pt')
model.load_state_dict(torch.load(my_file))
model.eval()
    
on_adv_acc = accuracy2(model, onadv_test_loader, device)

accuracy: 0.3128834355828221


In [110]:
save_data(onadv_loader, './simulation/HAR_UCI/', filename='on_train_loader.pkl')
save_data(onadv_test_loader, './simulation/HAR_UCI/', filename='on_test_loader.pkl')

In [13]:
onadv_loader = load_data('./simulation/HAR_UCI/on_train_loader.pkl')
onadv_test_loader = load_data( './simulation/HAR_UCI/on_test_loader.pkl')

In [16]:
def trades_iter(model, optimizer, criterion, data_loader, device, comp_ratio=0., step_size=0.01, epsilon=0.07, perturb_steps=7, beta=1.):
    model.train()
    iteration_loss = AverageMeter()
    for i, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.long().to(device)

        optimizer.zero_grad()
        
        
        # calculate robust loss
        loss = trades_loss(model=model,
                           x_natural=X,
                           y=y,
                           optimizer=optimizer,
                           device=device,
                           step_size=step_size,
                           epsilon=epsilon,
                           perturb_steps=perturb_steps,
                           beta=beta)
        
        
        iteration_loss.update(loss.item(), X.size(0))
        
        loss.backward()
        optimizer.step()

        if comp_ratio > 0:
            idxs, lams = structured_har(model, comp_ratio)
    if comp_ratio > 0:
        idxs, lams = structured_har(model, comp_ratio)
    return iteration_loss.avg

def dual_train_iter(model, adv, optimizer, criterion, on_loader, train_loader, device, comp_ratio=0.):
    model.train()
    
    iteration_loss = AverageMeter()
    for i, ((X, y), (X2, y2)) in enumerate(zip(train_loader, on_loader)):
        X, y = X.to(device), y.long().to(device)
        X2, y2 = X2.to(device), y2.long().to(device)
        X1 = adv.perturb(X, y)
        
        out = model(X1)
        out2 = model(X2)
        
        out = torch.cat((out, out2))
        ys = torch.cat((y, y2))

        l = criterion(out, ys)

        iteration_loss.update(l.item(), ys.size(0))

        optimizer.zero_grad()
        l.backward()
        optimizer.step()

        if comp_ratio > 0:
            idxs, lams = structured_har(model, comp_ratio)
    if comp_ratio > 0:
        idxs, lams = structured_har(model, comp_ratio)
    return iteration_loss.avg



def adv_train_iter(model, adv, optimizer, criterion, data_loader, device, comp_ratio=0.):
    model.train()
    
    iteration_loss = AverageMeter()
    for i, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.long().to(device)
        
        adv_x = adv.perturb(X, y.long())

        output = model(adv_x)

        loss = criterion(output, y.long())

        iteration_loss.update(loss.item(), X.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if comp_ratio > 0:
            idxs, lams = structured_har(model, comp_ratio)
    if comp_ratio > 0:
        idxs, lams = structured_har(model, comp_ratio)
    return iteration_loss.avg

def train_iter(model, optimizer, criterion, data_loader, device, comp_ratio=0.):
    model.train()
    iteration_loss = AverageMeter()
    for i, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        output = model(X)

        loss = criterion(output, y.long())

        iteration_loss.update(loss.item(), X.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if comp_ratio > 0:
            idxs, lams = structured_har(model, comp_ratio)
#             print(model.conv1.weight.data)
    if comp_ratio > 0:
        idxs, lams = structured_har(model, comp_ratio)
    return iteration_loss.avg

def test_iter(model, data_loader, device, check_adv=False, epsilon=0.3, alpha=0.0073, k=7):
    model.eval()
    normal_acc = AverageMeter()

    if check_adv:
        adv = LinfPGDAttack(model, epsilon=epsilon, alpha=alpha, k=k)
        off_acc = AverageMeter()

    for i, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        output = model(X)

        out = F.softmax(output, dim=1)
        _, predicted = out.max(1)
        idx = predicted.eq(y)

        acc = idx.sum().item() / X.size(0)
        normal_acc.update(acc)

        if check_adv:
            adv_x = adv.perturb(X, y.long())

            out = model(adv_x)
            out = F.softmax(out, dim=1)
            _, predicted = out.max(1)
            idx = predicted.eq(y)

            acc = idx.sum().item() / X.size(0)
            off_acc.update(acc)

    if check_adv:
        return normal_acc.avg, off_acc.avg
    else:
        return normal_acc.avg

In [23]:
def trades_comp(comp_ratio, retrain=False, EPOCHs=15, epsilon=0.07, k=7, alpha=0.01):
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    
    model = HARClassifier()
    model.to(device)
    
    my_file = Path('./simulation/HAR_UCI/comp{}-trades-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), 19))
    if my_file.exists() and not retrain:
        model.load_state_dict(torch.load(my_file))
        model.eval()
    
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
        
    else:
        model.load_state_dict(torch.load(os.path.join(model_dir, 'comp0_0-model-epoch{}.pt'.format(99))))
    
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
        scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
        criterion = nn.CrossEntropyLoss()

        for epoch in tqdm(range(EPOCHs)):
            train_loss = trades_iter(model, optimizer, criterion, train_loader, device, comp_ratio, step_size=alpha, epsilon=epsilon, perturb_steps=k, beta=1.)
#             train_loss = train_iter(model, optimizer, criterion, train_loader, device, comp_ratio)
            val_acc = test_iter(model, test_loader, device)
            scheduler.step()
            
            if (epoch + 1) % 5 == 0:
                print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
                torch.save(model.state_dict(),
                               os.path.join(model_dir,
                                            'comp{}-trades-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
            
        
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
    benign_acc, benign_correct, benign_total = accuracy2(model, test_loader, device)
    off_adv_acc, _, _ = accuracy2(model, off_test_loader, device)
    on_adv_acc, on_correct, on_total = accuracy2(model, onadv_test_loader, device)
    
#     normal = (on_correct + benign_correct) / (benign_total + on_total)
    normal = (on_total/benign_total * benign_correct + on_correct) / (2 * on_total)
    print(normal)
    
    return benign_acc, off_adv_acc, on_adv_acc, normal
    
    



def offadv_comp(comp_ratio, retrain=False, EPOCHs=15, epsilon=0.07, k=7, alpha=0.01):
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    model = HARClassifier()
    model.to(device)
    
    my_file = Path('./simulation/HAR_UCI/comp{}-offadv-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), 19))
    if my_file.exists() and not retrain:
        model.load_state_dict(torch.load(my_file))
        model.eval()
    
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
        
    else:
        model.load_state_dict(torch.load(os.path.join(model_dir, 'comp0_0-model-epoch{}.pt'.format(99))))
        
        adv = LinfPGDAttack(model, epsilon=epsilon, alpha=alpha, k=k)
    
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
        scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
        criterion = nn.CrossEntropyLoss()

        for epoch in tqdm(range(EPOCHs)):
            train_loss = adv_train_iter(model, adv, optimizer, criterion, train_loader, device, comp_ratio=comp_ratio)
            val_acc = test_iter(model, test_loader, device)
            scheduler.step()
            
            if (epoch + 1) % 5 == 0:
                print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
                torch.save(model.state_dict(),
                               os.path.join(model_dir,
                                            'comp{}-offadv-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
            
        
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
    benign_acc, benign_correct, benign_total = accuracy2(model, test_loader, device)
    off_adv_acc, _, _ = accuracy2(model, off_test_loader, device)
    on_adv_acc, on_correct, on_total = accuracy2(model, onadv_test_loader, device)
    
#     normal = (on_correct + benign_correct) / (benign_total + on_total)
    normal = (on_total/benign_total * benign_correct + on_correct) / (2 * on_total)
    print(normal)
    
    return benign_acc, off_adv_acc, on_adv_acc, normal
    
def dualadv_comp(comp_ratio, onadv_loader, retrain=False, EPOCHs=15, epsilon=0.07, k=7, alpha=0.01):
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    model = HARClassifier()
    model.to(device)
    
    my_file = Path('./simulation/HAR_UCI/comp{}-dualadv-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), 19))
    if my_file.exists() and not retrain:
        model.load_state_dict(torch.load(my_file))
        model.eval()
    
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
        
    else:
        model.load_state_dict(torch.load(os.path.join(model_dir, 'comp0_0-model-epoch{}.pt'.format(99))))
        
        adv = LinfPGDAttack(model, epsilon=epsilon, alpha=alpha, k=k)
    
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
        scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
        criterion = nn.CrossEntropyLoss()

        for epoch in tqdm(range(EPOCHs)):
            train_loss = dual_train_iter(model, adv, optimizer, criterion, onadv_loader, train_loader, device, comp_ratio)
            val_acc = test_iter(model, test_loader, device)
            scheduler.step()

            if (epoch + 1) % 5 == 0:
                print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
                torch.save(model.state_dict(),
                               os.path.join(model_dir,
                                            'comp{}-dualadv-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
            
        
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
    benign_acc, benign_correct, benign_total = accuracy2(model, test_loader, device)
    off_adv_acc, _, _ = accuracy2(model, off_test_loader, device)
    on_adv_acc, on_correct, on_total = accuracy2(model, onadv_test_loader, device)
    
#     normal = (on_correct + benign_correct) / (benign_total + on_total)
    normal = (on_total/benign_total * benign_correct + on_correct) / (2 * on_total)
    print(normal)
    
    return benign_acc, off_adv_acc, on_adv_acc, normal

def onadv_comp(comp_ratio, onadv_loader, retrain=False, EPOCHs=15, save=True):
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    model = HARClassifier()
    model.to(device)
    
    my_file = Path('./simulation/HAR_UCI/comp{}-onadv-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), 19))
    if my_file.exists() and not retrain:
        model.load_state_dict(torch.load(my_file))
        model.eval()
    
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
        
    else:
        model.load_state_dict(torch.load(os.path.join(model_dir, 'comp0_0-model-epoch{}.pt'.format(99))))
    
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
        scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
        criterion = nn.CrossEntropyLoss()

        for epoch in tqdm(range(EPOCHs)):
            train_loss = train_iter(model, optimizer, criterion, onadv_loader, device, comp_ratio)
            val_acc = test_iter(model, test_loader, device)
            scheduler.step()

            if (epoch + 1) % 5 == 0 and save:
                print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
                torch.save(model.state_dict(),
                               os.path.join(model_dir,
                                            'comp{}-onadv-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
            
    benign_acc, benign_correct, benign_total = accuracy2(model, test_loader, device)
    off_adv_acc, _, _ = accuracy2(model, off_test_loader, device)
    on_adv_acc, on_correct, on_total = accuracy2(model, onadv_test_loader, device)
    
#     normal = (on_correct + benign_correct) / (benign_total + on_total)
    normal = (on_total/benign_total * benign_correct + on_correct) / (2 * on_total)
    print(normal)
    
    return benign_acc, off_adv_acc, on_adv_acc, normal



def benign_comp(comp_ratio, retrain=False, EPOCHs=15):
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    
    model = HARClassifier()
    model.to(device)
    
    my_file = Path('./simulation/HAR_UCI/comp{}-model-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), 19))
    print(my_file)
    if my_file.exists() and not retrain:
        model.load_state_dict(torch.load(my_file))
        model.eval()
        
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
    if comp_ratio == 0.:
        model.load_state_dict(torch.load(os.path.join(model_dir, 'comp0_0-model-epoch{}.pt'.format(99))))
        model.eval()
    
    else:
        model.load_state_dict(torch.load(os.path.join(model_dir, 'comp0_0-model-epoch{}.pt'.format(99))))
    
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
        scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
        criterion = nn.CrossEntropyLoss()

        for epoch in tqdm(range(EPOCHs)):
            train_loss = train_iter(model, optimizer, criterion, train_loader, device, comp_ratio)
            val_acc = test_iter(model, test_loader, device)
            scheduler.step()

            if (epoch + 1) % 5 == 0:
                print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
                torch.save(model.state_dict(),
                               os.path.join(model_dir,
                                            'comp{}-model-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
            
    benign_acc, benign_correct, benign_total = accuracy2(model, test_loader, device)
    off_adv_acc, _, _ = accuracy2(model, off_test_loader, device)
    on_adv_acc, on_correct, on_total = accuracy2(model, onadv_test_loader, device)
    
#     normal = (on_correct + benign_correct) / (benign_total + on_total)
    normal = (on_total/benign_total * benign_correct + on_correct) / (2 * on_total)
    print(normal)
    
    return benign_acc, off_adv_acc, on_adv_acc, normal

In [21]:
Path('./simulation/HAR_UCI/model-epoch49.pt')

comp_ratio = 1.

model = resnet18_1d(output_len=6, input_channel=9)
model.to(device)

my_file = Path('./simulation/HAR_UCI/comp100_0-resnet-epoch19.pt')
model.load_state_dict(torch.load(my_file))
model.eval()
    
on_adv_acc = accuracy2(model, test_loader, device)

accuracy: 0.9002375296912114


In [17]:
a = model.conv1.weight.data

In [18]:
a[a != 0]

tensor([ 1.2182e-01,  3.4760e-02,  2.1399e-01,  1.0390e-01,  8.9060e-02,
        -1.2481e-01,  1.3397e-01, -2.6097e-01, -1.1569e-01, -3.6186e-02,
         3.1360e-02, -6.8807e-03,  1.2096e-01,  1.4558e-01, -2.6061e-01,
        -2.0990e-01, -3.4287e-01, -2.0954e-01, -5.6565e-02,  1.0485e-01,
         2.6357e-01, -1.0278e-01, -4.2792e-02, -1.7414e-01, -8.6568e-02,
        -1.0691e-01, -6.9222e-02,  3.3895e-01,  4.0046e-01, -1.7424e-02,
        -1.5126e-01, -1.3848e-01, -1.0038e-01, -1.6331e-03, -6.9561e-02,
        -6.8052e-04,  7.5151e-02,  2.3516e-01, -2.3290e-01, -1.5466e-01,
        -8.1470e-02,  2.0137e-01,  6.1631e-01,  5.2196e-01,  5.5525e-01,
         3.8745e-01,  4.3791e-01,  4.2734e-01,  4.3617e-01, -6.1679e-01,
         1.8969e-01,  3.4669e-01,  1.9619e-01,  3.7165e-01,  4.2038e-01,
         5.8306e-01, -1.0666e+00, -1.0356e-02, -8.6998e-03, -4.5606e-02,
         1.8110e-01,  4.2872e-01,  5.6085e-01], device='cuda:4')

In [11]:
def trades_comp(comp_ratio, retrain=False, EPOCHs=15, epsilon=0.07, k=7, alpha=0.01):
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    
    model = resnet18_1d(output_len=6, input_channel=9)
    model.to(device)
    
    my_file = Path('./simulation/HAR_UCI/comp{}-trades-resnet-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), 19))
    if my_file.exists() and not retrain:
        model.load_state_dict(torch.load(my_file))
        model.eval()
    
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
        
    else:
        model.load_state_dict(torch.load(os.path.join(model_dir, 'model-epoch49.pt')))
    
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
        scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
        criterion = nn.CrossEntropyLoss()

        for epoch in tqdm(range(EPOCHs)):
            train_loss = trades_iter(model, optimizer, criterion, train_loader, device, comp_ratio, step_size=alpha, epsilon=epsilon, perturb_steps=k, beta=1.)
#             train_loss = train_iter(model, optimizer, criterion, train_loader, device, comp_ratio)
            val_acc = test_iter(model, test_loader, device)
            scheduler.step()
            
            if (epoch + 1) % 5 == 0:
                print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
                torch.save(model.state_dict(),
                               os.path.join(model_dir,
                                            'comp{}-trades-resnet-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
            
        
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
    benign_acc, benign_correct, benign_total = accuracy2(model, test_loader, device)
    off_adv_acc, _, _ = accuracy2(model, off_test_loader, device)
    on_adv_acc, on_correct, on_total = accuracy2(model, onadv_test_loader, device)
    
#     normal = (on_correct + benign_correct) / (benign_total + on_total)
    normal = (on_total/benign_total * benign_correct + on_correct) / (2 * on_total)
    print(normal)
    
    return benign_acc, off_adv_acc, on_adv_acc, normal


def offadv_comp(comp_ratio, retrain=False, EPOCHs=15, epsilon=0.07, k=7, alpha=0.01):
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    model = resnet18_1d(output_len=6, input_channel=9)
    model.to(device)
    
    my_file = Path('./simulation/HAR_UCI/comp{}-offadv-resnet-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), 19))
    if my_file.exists() and not retrain:
        model.load_state_dict(torch.load(my_file))
        model.eval()
    
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
        
    else:
        model.load_state_dict(torch.load(os.path.join(model_dir, 'model-epoch49.pt')))
        
        adv = LinfPGDAttack(model, epsilon=epsilon, alpha=alpha, k=k)
    
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
        scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
        criterion = nn.CrossEntropyLoss()

        for epoch in tqdm(range(EPOCHs)):
            train_loss = adv_train_iter(model, adv, optimizer, criterion, train_loader, device, comp_ratio=comp_ratio)
            val_acc = test_iter(model, test_loader, device)
            scheduler.step()
            
            if (epoch + 1) % 5 == 0:
                print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
                torch.save(model.state_dict(),
                               os.path.join(model_dir,
                                            'comp{}-offadv-resnet-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
            
        
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
    benign_acc, benign_correct, benign_total = accuracy2(model, test_loader, device)
    off_adv_acc, _, _ = accuracy2(model, off_test_loader, device)
    on_adv_acc, on_correct, on_total = accuracy2(model, onadv_test_loader, device)
    
#     normal = (on_correct + benign_correct) / (benign_total + on_total)
    normal = (on_total/benign_total * benign_correct + on_correct) / (2 * on_total)
    print(normal)
    
    return benign_acc, off_adv_acc, on_adv_acc, normal
    
def dualadv_comp(comp_ratio, onadv_loader, retrain=False, EPOCHs=15, epsilon=0.07, k=7, alpha=0.01):
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    model = resnet18_1d(output_len=6, input_channel=9)
    model.to(device)
    
    my_file = Path('./simulation/HAR_UCI/comp{}-dualadv-resnet-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), 19))
    if my_file.exists() and not retrain:
        model.load_state_dict(torch.load(my_file))
        model.eval()
    
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
        
    else:
        model.load_state_dict(torch.load(os.path.join(model_dir, 'model-epoch49.pt')))
        
        adv = LinfPGDAttack(model, epsilon=epsilon, alpha=alpha, k=k)
    
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
        scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
        criterion = nn.CrossEntropyLoss()

        for epoch in tqdm(range(EPOCHs)):
            train_loss = dual_train_iter(model, adv, optimizer, criterion, onadv_loader, train_loader, device, comp_ratio)
            val_acc = test_iter(model, test_loader, device)
            scheduler.step()

            if (epoch + 1) % 5 == 0:
                print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
                torch.save(model.state_dict(),
                               os.path.join(model_dir,
                                            'comp{}-dualadv-resnet-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
            
        
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
    benign_acc, benign_correct, benign_total = accuracy2(model, test_loader, device)
    off_adv_acc, _, _ = accuracy2(model, off_test_loader, device)
    on_adv_acc, on_correct, on_total = accuracy2(model, onadv_test_loader, device)
    
#     normal = (on_correct + benign_correct) / (benign_total + on_total)
    normal = (on_total/benign_total * benign_correct + on_correct) / (2 * on_total)    

    print(normal)
    
    
    return benign_acc, off_adv_acc, on_adv_acc, normal

def onadv_comp(comp_ratio, onadv_loader, retrain=False, EPOCHs=15, save=True):
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    model = resnet18_1d(output_len=6, input_channel=9)
    model.to(device)
    
    my_file = Path('./simulation/HAR_UCI/comp{}-onadv-resnet-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), 19))
    if my_file.exists() and not retrain:
        model.load_state_dict(torch.load(my_file))
        model.eval()
    
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
        
    else:
        model.load_state_dict(torch.load(os.path.join(model_dir, 'model-epoch49.pt')))
    
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
        scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
        criterion = nn.CrossEntropyLoss()

        for epoch in tqdm(range(EPOCHs)):
            train_loss = train_iter(model, optimizer, criterion, onadv_loader, device, comp_ratio)
            val_acc = test_iter(model, test_loader, device)
            scheduler.step()

            if (epoch + 1) % 5 == 0 and save:
                print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
                torch.save(model.state_dict(),
                               os.path.join(model_dir,
                                            'comp{}-onadv-resnet-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
            
        
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
    benign_acc, benign_correct, benign_total = accuracy2(model, test_loader, device)
    off_adv_acc, _, _ = accuracy2(model, off_test_loader, device)
    on_adv_acc, on_correct, on_total = accuracy2(model, onadv_test_loader, device)
    
#     normal = (on_correct + benign_correct) / (benign_total + on_total)
    normal = (on_total/benign_total * benign_correct + on_correct) / (2 * on_total)
    
    print(normal)
    
    return benign_acc, off_adv_acc, on_adv_acc, normal


def benign_comp(comp_ratio, retrain=False, EPOCHs=15):
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    
    model = resnet18_1d(output_len=6, input_channel=9)
    model.to(device)
    
    my_file = Path('./simulation/HAR_UCI/comp{}-resnet-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), 19))
    print(my_file)
    
    if comp_ratio == 0:
        my_file = Path('./simulation/HAR_UCI/model-epoch49.pt')
    
    if my_file.exists() and not retrain:
        model.load_state_dict(torch.load(my_file))
        model.eval()
    
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
        
    else:
        model.load_state_dict(torch.load(os.path.join(model_dir, 'model-epoch49.pt')))
    
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
        scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
        criterion = nn.CrossEntropyLoss()

        for epoch in tqdm(range(EPOCHs)):
            train_loss = train_iter(model, optimizer, criterion, train_loader, device, comp_ratio)
            val_acc = test_iter(model, test_loader, device)
            scheduler.step()

            if (epoch + 1) % 5 == 0:
                print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
                torch.save(model.state_dict(),
                               os.path.join(model_dir,
                                            'comp{}-resnet-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
            
        
#         benign_acc, off_adv_acc, correct_list, correct_label, adv_list = accuracy(model, test_loader, device, epsilon=0.07, k=7, alpha=0.01)
#         on_adv_acc = accuracy2(model, onadv_test_loader, device)
    benign_acc, benign_correct, benign_total = accuracy2(model, test_loader, device)
    off_adv_acc, _, _ = accuracy2(model, off_test_loader, device)
    on_adv_acc, on_correct, on_total = accuracy2(model, onadv_test_loader, device)
    
#     normal = (on_correct + benign_correct) / (benign_total + on_total)
    normal = (on_total/benign_total * benign_correct + on_correct) / (2 * on_total)
    
    print(normal)
    
    return benign_acc, off_adv_acc, on_adv_acc, normal

In [200]:
be, off, on = offadv_comp(0.5, retrain=True, EPOCHs=1)

  0%|          | 0/1 [00:00<?, ?it/s]

benign accuracy: 0.7553444180522565	adversarial accuracy: 0.5890736342042755
accuracy: 0.0835509138381201


In [201]:
be, off, on = onadv_comp(0.5, onadv_loader, retrain=True, EPOCHs=1)

  0%|          | 0/1 [00:00<?, ?it/s]

benign accuracy: 0.8995588734306074	adversarial accuracy: 0.19409569053274517
accuracy: 0.3785900783289817


In [224]:
be, off, on = benign_comp(0., retrain=False, EPOCHs=20)
be, off, on = onadv_comp(0., onadv_loader, retrain=False, EPOCHs=20)
be, off, on = offadv_comp(0., retrain=False, EPOCHs=20)
be, off, on = dualadv_comp(0., onadv_loader, retrain=False, EPOCHs=20)
be, off, on = trades_comp(0., retrain=False, EPOCHs=20)


#     normal_acc, off_acc, on_acc = benign_comp(comp_ratio, retrain=retrain)
#     normal['normal'].append(normal_acc)
#     normal['off'].append(off_acc)
#     normal['on'].append(on_acc)
#     # onadv train comp
#     normal_acc, off_acc, on_acc = onadv_comp(comp_ratio, onadv_loader, retrain=True)
#     onadv['normal'].append(normal_acc)
#     onadv['off'].append(off_acc)
#     onadv['on'].append(on_acc)
#     # offadv train comp
#     normal_acc, off_acc, on_acc = offadv_comp(comp_ratio, retrain=True)
#     offadv['normal'].append(normal_acc)
#     offadv['off'].append(off_acc)
#     offadv['on'].append(on_acc)
#     # onoffadv train comp
#     normal_acc, off_acc, on_acc = dualadv_comp(comp_ratio, onadv_loader, retrain=True)
#     dualadv['normal'].append(normal_acc)
#     dualadv['off'].append(off_acc)
#     dualadv['on'].append(on_acc)
#     # trades train comp
#     normal_acc, off_acc, on_acc = trades_comp(comp_ratio, retrain=True)
#     trades['normal'].append(normal_acc)
#     trades['off'].append(off_acc)
#     trades['on'].append(on_acc)

accuracy: 0.9256871394638616
accuracy: 0.0
accuracy: 0.0
accuracy: 0.9436715303698676
accuracy: 0.0246965257429887
accuracy: 0.41604010025062654
accuracy: 0.8520529351883271
accuracy: 0.8656341565508581
accuracy: 0.11278195488721804
accuracy: 0.8707159823549372
accuracy: 0.8668899120971117
accuracy: 0.11278195488721804
accuracy: 0.9409569053274517
accuracy: 0.5429049811636668
accuracy: 0.0


In [223]:
on

0.0

### 1d Resnet train

In [176]:
train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')

model = resnet18_1d(output_len=6, input_channel=9)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
criterion = nn.CrossEntropyLoss()

for epoch in tqdm(range(50)):
    train_loss = train_iter(model, optimizer, criterion, train_loader, device, 0)
    val_acc = test_iter(model, test_loader, device)
    scheduler.step()

    if (epoch + 1) % 5 == 0:
        print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
        torch.save(model.state_dict(),
                       os.path.join(model_dir,
                                    'model-epoch{}.pt'.format(epoch)))

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 5	Train loss: 0.0911	Validation accuracy: 0.9203
Epoch 10	Train loss: 0.0844	Validation accuracy: 0.9237
Epoch 15	Train loss: 0.0840	Validation accuracy: 0.9203
Epoch 20	Train loss: 0.0754	Validation accuracy: 0.9333
Epoch 25	Train loss: 0.0712	Validation accuracy: 0.9330
Epoch 30	Train loss: 0.0591	Validation accuracy: 0.9320
Epoch 35	Train loss: 0.0584	Validation accuracy: 0.9243
Epoch 40	Train loss: 0.0677	Validation accuracy: 0.9303
Epoch 45	Train loss: 0.0523	Validation accuracy: 0.9287
Epoch 50	Train loss: 0.0488	Validation accuracy: 0.9270


In [ ]:
train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')

model = resnet18_1d(output_len=6, input_channel=9)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
scheduler = MultiStepLR(optimizer, milestones=[55, 75, 90], gamma=0.1)
criterion = nn.CrossEntropyLoss()

for epoch in tqdm(range(50)):
    train_loss = train_iter(model, optimizer, criterion, train_loader, device, 0.5)
    val_acc = test_iter(model, test_loader, device)
    scheduler.step()

    if (epoch + 1) % 5 == 0:
        print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch + 1, train_loss, val_acc))
        torch.save(model.state_dict(),
                       os.path.join(model_dir,
                                    'comp{}-resnet-epoch{}.pt'.format(epoch)))

In [25]:
normal = {
    'normal': [],
    'off': [],
    'on': [],
    'onnormal': []
}

onadv = {
    'normal': [],
    'off': [],
    'on': [],
    'onnormal': []
}

offadv = {
    'normal': [],
    'off': [],
    'on': [],
    'onnormal': []
}

dualadv = {
    'normal': [],
    'off': [],
    'on': [],
    'onnormal': []
}

trades = {
    'normal': [],
    'off': [],
    'on': [],
    'onnormal': []
}

# ratio_list = [0., 0.3, 0.5, 0.7, 0.9, 0.95, 0.99, 0.995, 0.999]
ratio_list = [0., 0.3, 0.5, 0.7, 0.9, 0.95, 0.99, 0.995, 0.999, 1.]

for comp_ratio in ratio_list:
    retrain = False
    print('compression ratio of {:.3f}'.format(comp_ratio))
    # benign comp
    
#     if comp_ratio == 1.:
#         retrain = False
#     else:
    retrain = True
    
    print('normal')
    normal_acc, off_acc, on_acc, onnormal = benign_comp(comp_ratio, retrain=retrain, EPOCHs=20)
    normal['normal'].append(normal_acc)
    normal['off'].append(off_acc)
    normal['on'].append(on_acc)
    normal['onnormal'].append(onnormal)
    # onadv train comp
    print('on')
    normal_acc, off_acc, on_acc, onnormal = onadv_comp(comp_ratio, onadv_loader, retrain=retrain, EPOCHs=20)
    onadv['normal'].append(normal_acc)
    onadv['off'].append(off_acc)
    onadv['on'].append(on_acc)
    onadv['onnormal'].append(onnormal)
    # offadv train comp
    print('off')
    normal_acc, off_acc, on_acc, onnormal = offadv_comp(comp_ratio, retrain=retrain, EPOCHs=20)
    offadv['normal'].append(normal_acc)
    offadv['off'].append(off_acc)
    offadv['on'].append(on_acc)
    offadv['onnormal'].append(onnormal)
    # onoffadv train comp
    print('dual')
    normal_acc, off_acc, on_acc, onnormal = dualadv_comp(comp_ratio, onadv_loader, retrain=retrain, EPOCHs=20)
    dualadv['normal'].append(normal_acc)
    dualadv['off'].append(off_acc)
    dualadv['on'].append(on_acc)
    dualadv['onnormal'].append(onnormal)
    # trades train comp
    print('trades')
    normal_acc, off_acc, on_acc, onnormal = trades_comp(comp_ratio, retrain=retrain, EPOCHs=20)
    trades['normal'].append(normal_acc)
    trades['off'].append(off_acc)
    trades['on'].append(on_acc)
    trades['onnormal'].append(onnormal)

compression ratio of 0.000
normal
simulation/HAR_UCI/comp0_0-model-epoch19.pt
accuracy: 0.9097387173396675
accuracy: 0.0
accuracy: 0.0
0.4548693586698337
on


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 5	Train loss: 0.2381	Validation accuracy: 0.9067
Epoch 10	Train loss: 0.2291	Validation accuracy: 0.9120
Epoch 15	Train loss: 0.2222	Validation accuracy: 0.9087
Epoch 20	Train loss: 0.2198	Validation accuracy: 0.9093
accuracy: 0.9077027485578555
accuracy: 0.3169456066945607
accuracy: 0.4051948051948052
0.6564487768763303
off


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 5	Train loss: 0.3944	Validation accuracy: 0.8563
Epoch 10	Train loss: 0.3035	Validation accuracy: 0.8613
Epoch 15	Train loss: 0.2635	Validation accuracy: 0.8557
Epoch 20	Train loss: 0.2502	Validation accuracy: 0.8633
accuracy: 0.8608754665761792
accuracy: 0.7583682008368201
accuracy: 0.12207792207792208
0.4914766943270506
dual


...

Epoch 5	Train loss: 1.7842	Validation accuracy: 0.1790
Epoch 10	Train loss: 1.7842	Validation accuracy: 0.1790
Epoch 15	Train loss: 1.7844	Validation accuracy: 0.1773
Epoch 20	Train loss: 1.7843	Validation accuracy: 0.1790
accuracy: 0.18221920597217509
accuracy: 0.0
accuracy: 0.11428571428571428
0.1482524601289447
dual


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 5	Train loss: 1.7855	Validation accuracy: 0.1790
Epoch 10	Train loss: 1.7855	Validation accuracy: 0.1790
Epoch 15	Train loss: 1.7855	Validation accuracy: 0.1790
Epoch 20	Train loss: 1.7852	Validation accuracy: 0.1790
accuracy: 0.18221920597217509
accuracy: 0.0
accuracy: 0.11428571428571428
0.1482524601289447
trades


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 5	Train loss: 1.7842	Validation accuracy: 0.1790
Epoch 10	Train loss: 1.7843	Validation accuracy: 0.1790
Epoch 15	Train loss: 1.7842	Validation accuracy: 0.1790
Epoch 20	Train loss: 1.7841	Validation accuracy: 0.1790
accuracy: 0.18221920597217509
accuracy: 0.0
accuracy: 0.11428571428571428
0.1482524601289447


In [26]:
total = {
    'normal_train': normal,
    'onadv_train': onadv,
    'offadv_train': offadv,
    'dualadv_train': dualadv,
    'trades_train': trades
}

save_data(total, './simulation/HAR_UCI/', filename='inference_result6.pkl')

In [74]:
model_dir = './simulation/HAR_UCI/'
model_file = os.path.join(model_dir, 'comp0_0-model-epoch{}.pt'.format(99))

state_dict = torch.load(model_file)

In [73]:
'./simulation/HAR_UCI/comp0_0-model-epoch99.pt'

'./simulation/HAR_UCI/comp0_0-model-epoch99.pt'

In [ ]:
def test_har(comp_ratio, device, model_dir='./simulation/HAR_UCI/'):
    state_dict = torch.load(os.path.join(model_dir,
                                    'comp{}-model-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
    
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    
    model = HARClassifier()
    model.to(device)
    model.load(state_dict)
    
    acc, off_acc = test_iter(model, test_loader, device, check_adv=True)

In [14]:
acc, off_acc

(0.9113333333333333, 0.32731205673758873)

In [12]:
model_dir='./simulation/HAR_UCI/'
device = 'cuda:%d'%4
state_dict = torch.load(os.path.join(model_dir,
                                    'comp0_0-model-epoch{}.pt'.format(99)))

train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')

model = HARClassifier()
model.to(device)
model.load_state_dict(state_dict)

acc, off_acc = test_iter(model, test_loader, device, check_adv=True)

In [5]:
def test_iter(model, data_loader, device, check_adv=False, epsilon=0.3, alpha=0.0073, k=7):
    
    model.eval()
    normal_acc = AverageMeter()

    if check_adv:
        adv = LinfPGDAttack(model, epsilon=epsilon, alpha=alpha, k=k)
        off_acc = AverageMeter()

    for i, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        output = model(X)

        out = F.softmax(output, dim=1)
        _, predicted = out.max(1)
        idx = predicted.eq(y)

        acc = idx.sum().item() / X.size(0)
        normal_acc.update(acc)

        if check_adv:
            adv_x = adv.perturb(X, y.long())

            out = model(adv_x)
            out = F.softmax(out, dim=1)
            _, predicted = out.max(1)
            idx = predicted.eq(y)

            acc = idx.sum().item() / X.size(0)
            off_acc.update(acc)

    if check_adv:
        return normal_acc.avg, off_acc.avg
    else:
        return normal_acc.avg